Use K-means clustering to predict returns of futures traded on global markets.

Import Packages

In [1]:
import sys
!{sys.executable} -m pip install scikit-learn
import yfinance as yf
import pandas as pd
import numpy as np
import plotnine

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Download Futures Data from yfinance:
USD, Gold, Silver, Copper, Brent Crude Oil, Natural Gas, Corn, Soybean, S&P 500, Nasdaq 100, Russell 2000

Store only closing data in the df we will use

In [2]:
tickers = ['DX=F', 'GC=F', 'SI=F', 'HG=F', 'BZ=F', 'NG=F', 'ZC=F', 'ZS=F', 'ES=F', 'NQ=F']
ticker_to_name = {
    "BZ=F": "Crude",
    "GC=F": "Gold",
    "NG=F": "Nat_Gas",
    "SI=F": "Silver",
    "ZC=F": "Corn",
    "ZS=F": "Soybeans",
    "ES=F": "S&P_500",
    "NQ=F": "Nasdaq_100",
    "DX=F": "USD",
    "HG=F": "Copper"
}

full_data = yf.download(tickers, start = '2000-01-01', end='2024-12-31')
full_data.columns = full_data.columns.set_levels([ticker_to_name.get(t, t) for t in full_data.columns.levels[1]], level=1)

close_data = full_data["Close"]

C:\Users\tcrow\AppData\Local\Temp\ipykernel_3276\1182904538.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  10 of 10 completed


Describe Futures Close Data Gathered

In [3]:
close_data.describe()

Ticker,Crude,USD,S&P_500,Gold,Copper,Nat_Gas,Nasdaq_100,Silver,Corn,Soybeans
count,4332.000000,6193.000000,6133.000000,6105.000000,6110.000000,6111.000000,6133.000000,6107.000000,6118.000000,6110.000000
mean,78.627620,92.340932,2128.468195,1160.788845,2.761719,4.428152,5297.366104,17.044722,410.785673,1006.032774
std,24.333126,11.296524,1258.958680,580.298316,1.120774,2.241100,5056.885653,8.499005,160.090006,332.053084
min,19.330000,71.304001,676.000000,255.100006,0.604000,1.482000,809.500000,4.026000,174.750000,418.500000
25%,60.057501,82.404999,1193.000000,631.700012,2.093500,2.776000,1693.000000,11.477500,310.562500,779.250000
50%,76.754997,92.411003,1513.250000,1233.900024,2.990500,3.786000,2769.750000,16.747999,375.250000,979.750000
75%,100.324999,99.589996,2781.000000,1615.199951,3.594000,5.597000,7215.000000,22.682000,501.937500,1295.687500
max,146.080002,121.209999,6099.000000,2788.500000,5.119000,15.378000,22110.250000,48.584000,831.250000,1771.000000


In [4]:
close_data.head()

Ticker,Crude,USD,S&P_500,Gold,Copper,Nat_Gas,Nasdaq_100,Silver,Corn,Soybeans
Date,,,,,,,,,,
2000-06-22,NaN,107.400002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-23,NaN,107.720001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-26,NaN,107.769997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-27,NaN,106.980003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-28,NaN,107.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
close_data.tail()

Ticker,Crude,USD,S&P_500,Gold,Copper,Nat_Gas,Nasdaq_100,Silver,Corn,Soybeans
Date,,,,,,,,,,
2024-12-23,72.629997,107.806999,6036.00,2612.300049,4.0230,3.656,21753.25,29.888000,447.75,969.50
2024-12-24,73.580002,108.023003,6098.00,2620.000000,4.0495,3.946,22028.50,29.974001,448.50,975.25
2024-12-26,73.260002,107.892998,6095.25,2638.800049,4.0720,3.715,22008.00,30.047001,453.75,988.00
2024-12-27,74.169998,107.792000,6027.00,2617.199951,4.0625,3.514,21698.50,29.655001,454.00,980.00
2024-12-30,74.389999,107.932999,5958.75,2606.100098,4.0455,3.936,21416.25,29.106001,452.25,982.00


In [6]:
close_data.shape

(6209, 10)

Lets create our derived variabels: Rolling averages (3 day, 7 day, 10 day, 14 day, 30 day), std deviations

In [ ]:
column_names = ["USD", "Crude", "Nat_Gas", "Gold", "Silver", "Copper", "Corn", "Soybeans", "S&P_500", "Nasdaq_100"]
windows = [3, 7, 10, 14, 30]
sd_windows = [30, 90, 180]
roi

# Close block as MultiIndex
base = close_data.reindex(columns=column_names).copy()
base.columns = pd.MultiIndex.from_product([["Close"], column_names])

# SMA blocks as MultiIndex, one per window
sma_frames = {}
for w in windows:
    sma = close_data.reindex(columns=column_names).rolling(window=w).mean()
    sma.columns = pd.MultiIndex.from_product([[f"{w}-day SMA"], column_names])
    sma_frames[w] = sma

# SD blocks, longer windows
sd_frames = {}
for w in sd_windows:
    sd = close_data.rolling(window=w).std()
    sd.columns = pd.MultiIndex.from_product([[f"{w}-day SD"], column_names])
    sd_frames[w] = sd

# Add 1 month, 3 month, and 6 month returns to dataset


# Add volume traded


# Concatenate and round
data = pd.concat([base] + [sma_frames[w] for w in windows] + [sd_frames[w] for w in sd_windows], axis=1).round(3)

data.tail()

Close                                                         \
                USD  Crude Nat_Gas    Gold  Silver Copper    Corn Soybeans   
Date                                                                         
2024-12-23  107.807  72.63   3.656  2612.3  29.888  4.023  447.75   969.50   
2024-12-24  108.023  73.58   3.946  2620.0  29.974  4.049  448.50   975.25   
2024-12-26  107.893  73.26   3.715  2638.8  30.047  4.072  453.75   988.00   
2024-12-27  107.792  74.17   3.514  2617.2  29.655  4.062  454.00   980.00   
2024-12-30  107.933  74.39   3.936  2606.1  29.106  4.045  452.25   982.00   

                                ... 180-day SD                           \
            S&P_500 Nasdaq_100  ...        USD  Crude  Nat_Gas     Gold   
Date                            ...                                       
2024-12-23  6036.00   21753.25  ...      5.597  1.922  292.879  145.285   
2024-12-24  6098.00   22028.50  ...      5.557  1.943  294.147  145.136   
2024-12-26  6095.25   22008.00  ...      5.504  1.962  294.940  144.979   
2024-12-27  6027.00   21698.50  ...      5.454  1.980  295.327  144.847   
2024-12-30  5958.75   21416.25  ...      5.392  1.995  294.574  144.662   

                                                                
           Silver Copper      Corn Soybeans S&P_500 Nasdaq_100  
Date                                                            
2024-12-23  0.251  0.459  1125.752    1.776  25.433     92.215  
2024-12-24  0.252  0.469  1134.898    1.769  25.512     92.161  
2024-12-26  0.252  0.476  1142.225    1.762  25.619     92.087  
2024-12-27  0.253  0.478  1147.896    1.757  25.741     92.074  
2024-12-30  0.254  0.486  1148.680    1.754  25.830     91.950  

[5 rows x 90 columns]

In [9]:
data.shape

(6209, 90)

In [15]:
data.describe()

Close                                                      \
               USD        Crude      Nat_Gas         Gold       Silver   
count  6193.000000  4332.000000  6111.000000  6105.000000  6107.000000   
mean     92.340932    78.627620     4.428152  1160.788845    17.044722   
std      11.296524    24.333126     2.241100   580.298317     8.499005   
min      71.304000    19.330000     1.482000   255.100000     4.026000   
25%      82.405000    60.057500     2.776000   631.700000    11.477500   
50%      92.411000    76.755000     3.786000  1233.900000    16.748000   
75%      99.590000   100.325000     5.597000  1615.200000    22.682000   
max     121.210000   146.080000    15.378000  2788.500000    48.584000   

                                                                         ...  \
            Copper         Corn     Soybeans      S&P_500    Nasdaq_100  ...   
count  6110.000000  6118.000000  6110.000000  6133.000000   6133.000000  ...   
mean      2.761719   410.785673  1006.032774  2128.468195   5297.366105  ...   
std       1.120776   160.090006   332.053084  1258.958679   5056.885653  ...   
min       0.604000   174.750000   418.500000   676.000000    809.500000  ...   
25%       2.093000   310.562500   779.250000  1193.000000   1693.000000  ...   
50%       2.990000   375.250000   979.750000  1513.250000   2769.750000  ...   
75%       3.594000   501.937500  1295.687500  2781.000000   7215.000000  ...   
max       5.119000   831.250000  1771.000000  6099.000000  22110.250000  ...   

        180-day SD                                                      \
               USD        Crude      Nat_Gas         Gold       Silver   
count  2703.000000  4577.000000  5078.000000  3990.000000  4196.000000   
mean      8.098950     2.144196   111.050934    71.614411     0.255091   
std       4.212237     0.954888    80.244930    30.821596     0.156443   
min       2.993000     0.480000    19.689000    19.598000     0.064000   
25%       5.135500     1.426000    57.089500    49.260000     0.152000   
50%       6.458000     1.977000    77.292000    66.457500     0.211000   
75%      10.062500     2.743000   151.555750    82.297250     0.333000   
max      21.672000     5.119000   358.258000   179.053000     0.974000   

                                                                        
            Copper         Corn     Soybeans      S&P_500   Nasdaq_100  
count  4398.000000  5078.000000  3990.000000  4076.000000  4257.000000  
mean      0.655108   381.939274     1.995565    47.673265    92.173265  
std       0.557664   393.763017     1.279339    29.756834    52.857945  
min       0.123000    43.143000     0.492000     9.536000    25.273000  
25%       0.308000   113.818000     1.077250    23.300500    47.339000  
50%       0.452000   192.708000     1.605500    37.891500    85.612000  
75%       0.716000   479.632000     2.520750    70.833250   120.377000  
max       2.652000  1634.071000     7.139000   126.718000   254.843000  

[8 rows x 90 columns]

Account for seasonality and trend if possible:

K-Means Clustering

Lets set our data to scale: